In [1]:
import openai
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 생성 (2.x)
client = openai.Client()

# 생성할 총 질문 수
TOTAL_QUESTIONS = 3


def build_student_summary(data):
    """생기부 JSON에서 면접용 요약 생성"""
    name = data["student_info"]["name"]

    # 수상 경력 요약
    awards_list = [a.get("award_name", "") for a in data.get("awards", [])]
    awards = ", ".join(awards_list) if awards_list else "없음"

    # 학업 관련 기록 정리
    academic_text = ""
    for rec in data.get("academic_records", []):
        if isinstance(rec, str):
            academic_text += rec + "\n"

    academic_text = academic_text[:2000]

    return f"""
이름: {name}
수상경력: {awards}

창의적 체험활동 / 교과 세부능력 주요 내용:
{academic_text}
"""


def start_ai_interview(target_student_data, training_context):
    target_summary = build_student_summary(target_student_data)


    # SYSTEM 프롬프트 — 규칙 강화
    system_prompt = f"""
    당신은 대한민국 대입 전문 면접관입니다.
    
    아래는 여러 학생들의 생활기록부 요약입니다.
    이 정보는 질문의 수준과 관점을 학습하기 위한 참고 자료입니다.
    이 학생들에게 직접 질문하지 마십시오.
    
    [학습용 생기부 모음]
    {training_context}
    
    --------------------------------------------------
    
    아래 학생이 실제 면접 대상입니다.
    모든 질문과 피드백은 반드시 이 학생의 생활기록부만을 기준으로 하십시오.
    
    [면접 대상 학생 생기부]
    {target_summary}
    
    면접 규칙:
    1) 첫 질문은 아래 형식으로만 출력한다:
       [첫 질문]
       질문 내용
    
    2) 두 번째부터 {TOTAL_QUESTIONS - 1}번째 질문까지는:
       [다음 질문]
       질문 내용
    
    3) {TOTAL_QUESTIONS}번째 질문(마지막)은:
       [마지막 질문]
       마지막 질문 1개만 출력
    
    4) 학생이 답변하면 반드시 아래 형식으로만 답한다:
       [피드백]
       학생 답변 평가 1~2문장
    
       [다음 질문] 또는 [마지막 질문]
       질문 내용
    
    5) 학습용 생기부의 학생을 언급하거나 비교하지 말 것.
    6) 규칙을 절대 어기지 말 것.
    """


    # 메시지 히스토리
    messages = [
        {"role": "system", "content": system_prompt}
    ]

    # -------------------------------------------------------
    # 첫 질문 요청 (피드백 절대 포함 금지)
    # -------------------------------------------------------
    first_question_prompt = f"""
생기부를 기반으로 첫 번째 질문을 출력하세요.

출력 형식:
[첫 질문]
질문 내용
"""
    messages.append({"role": "user", "content": first_question_prompt})

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=400,
    )

    first_question = resp.choices[0].message.content
    messages.append({"role": "assistant", "content": first_question})

    print("\n===== 🎤 AI 면접관 모드 시작 =====")
    print("종료하려면 exit 또는 quit 입력\n")
    print(first_question)

    # -------------------------------------------------------
    # 2번째 ~ 마지막 질문 루프
    # -------------------------------------------------------
    current_question_number = 2

    while current_question_number <= TOTAL_QUESTIONS:
        answer = input("\n[학생 답변] > ").strip()

        if answer.lower() in ("exit", "quit"):
            print("\n면접 연습을 종료합니다. 수고했어요! 🙌")
            break

        # 마지막 질문 여부 체크
        is_last = (current_question_number == TOTAL_QUESTIONS)

        follow_prompt = f"""
아래 학생의 답변을 평가하세요.

[학생 답변]
{answer}

출력 형식은 반드시 아래 중 하나:

{'[마지막 질문]' if is_last else '[다음 질문]'}

형식:
[피드백]
학생 답변 평가 1~2문장

{'[마지막 질문]' if is_last else '[다음 질문]'}
질문 내용
"""

        messages.append({"role": "user", "content": follow_prompt})

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=600,
        )

        content = resp.choices[0].message.content
        messages.append({"role": "assistant", "content": content})

        print("\n--------------------------------")
        print(content)

        # 마지막 질문 출력했으면 종료
        if is_last:
            print("\n✨ 모든 질문이 끝났습니다. 수고했어요! 🙌")
            break

        current_question_number += 1

        
def load_student_txt(filepath):
    """생활기록부 TXT 파일 → 내부 dict 구조로 변환"""
    with open(filepath, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    name = "이름 미상"
    awards = []
    academic_records = []

    section = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("이름"):
            name = line.split(":", 1)[1].strip()

        elif "수상경력" in line:
            section = "awards"

        elif "창의적 체험활동" in line or "교과 세부능력" in line:
            section = "academic"

        else:
            if section == "awards" and line.startswith("-"):
                awards.append({"award_name": line[1:].strip()})
            elif section == "academic":
                academic_records.append(line)

    return {
        "student_info": {"name": name},
        "awards": awards,
        "academic_records": academic_records,
    }


def build_training_context(recordings_dir, target_filename):
    """여러 학생 생기부를 학습용 컨텍스트로 합침 (타겟 제외)"""
    summaries = []

    for filename in os.listdir(recordings_dir):
        if not filename.endswith(".txt"):
            continue
        if filename == target_filename:
            continue

        filepath = os.path.join(recordings_dir, filename)
        data = load_student_txt(filepath)
        summary = build_student_summary(data)

        summaries.append(summary.strip())

    return "\n\n---\n\n".join(summaries)


RECORDINGS_DIR = "recordings_lite"
TARGET_FILE = "202315060_3학년_휴먼AI공학전공_서현승_정시_censored.txt"


if __name__ == "__main__":
    training_context = build_training_context(
        RECORDINGS_DIR,
        TARGET_FILE
    )

    target_path = os.path.join(RECORDINGS_DIR, TARGET_FILE)
    target_student_data = load_student_txt(target_path)

    # print("\n🎯 면접 대상:", target_student_data["student_info"]["name"])
    print("📚 학습용 생기부 수집 완료\n")

    start_ai_interview(
        target_student_data,
        training_context
    )

📚 학습용 생기부 수집 완료


===== 🎤 AI 면접관 모드 시작 =====
종료하려면 exit 또는 quit 입력

[첫 질문]
영어 교과반장으로서의 경험이 본인에게 어떤 의미였고, 어떤 점에서 성장을 느꼈는지 이야기해 주세요.


KeyboardInterrupt: Interrupted by user